This module is resposible for


*   Preprocessing



# Configuration and imports

In [ ]:
pip install spacy

In [ ]:
import csv
import pprint
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities
from itertools import chain
from collections import Counter
import spacy
import gensim, logging
import nltk
from time import time
import numpy as np
import pandas as pd
import string
from nltk.corpus import wordnet as wnpi
import pickle
from gensim.models import Word2Vec


import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer


#import techniques
#from techniques import *
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
txtSpellFile = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/corporaForSpellCorrection.txt'
txtSlangFile = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/slang.txt'
csvDestinationFileName = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/PreProcessedPang-Ver2-00-11-27.csv'
csvSourceFileName = '/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/Pang-Ver2-00-10-24.csv'


In [ ]:
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

Mounted at /MYDRIVE


# commpn functions

In [ ]:

""" Creates a dictionary with slangs and their equivalents and replaces them """
with open(txtSlangFile) as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])




In [ ]:
""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]


In [ ]:
def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    return text

def replaceURL(text):
    """ Replaces url address with "url" """
    #edited by seilsepour
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

def replaceAtUser(text):
    """ Replaces "@user" with "atUser" """
    #edited by seilsepour
    #text = re.sub('@[^\s]+','atUser',text)
    text = re.sub('@[^\s]+','',text)
    return text

def removeHashtagInFrontOfWord(text):
    """ Removes hastag in front of a word """
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def replaceMultiExclamationMark(text):
    """ Replaces repetitions of exlamation marks """
    #edited by seilsepour
    #text = re.sub(r"(\!)\1+", ' multiExclamation ', text)
    text = re.sub(r"(\!)\1+", '', text)
    return text

def replaceMultiQuestionMark(text):
    """ Replaces repetitions of question marks """
    #edited by seilsepour
    #text = re.sub(r"(\?)\1+", ' multiQuestion ', text)
    text = re.sub(r"(\?)\1+", '', text)
    return text

def replaceMultiStopMark(text):
    """ Replaces repetitions of stop marks """
    #edited by seilsepour
    #text = re.sub(r"(\.)\1+", ' multiStop ', text)
    text = re.sub(r"(\.)\1+", '', text)
    return text

def countMultiExclamationMarks(text):
    """ Replaces repetitions of exlamation marks """
    return text #len(re.findall(r"(\!)\1+", text))

def countMultiQuestionMarks(text):
  return text
  #return len(re.findall(r"(\?)\1+", text))

def countMultiStopMarks(text):
  return text
  #return len(re.findall(r"(\.)\1+", text))

def countElongated(text):
  return text
  #regex = re.compile(r"(.)\1{2}")
  #return len([word for word in text.split() if regex.search(word)])

def countAllCaps(text):
  return text
  #return len(re.findall("[A-Z0-9]{3,}", text))


In [ ]:
def countSlang(text):
    """ Input: a text, Output: how many slang words and a list of found slangs """
    slangCounter = 0
    slangsFound = []
    tokens = nltk.word_tokenize(text)
    for word in tokens:
        if word in slang_words:
            slangsFound.append(word)
            slangCounter += 1
    return 0, 0 #slangCounter, slangsFound

def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """
    #return word
    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    return text

def countEmoticons(text):
    """ Input: a text, Output: how many emoticons """
    return 0 #len(re.findall(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', text))


### Spell Correction begin ###
""" Spell Correction http://norvig.com/spell-correct.html """
def words(text): 
  return re.findall(r'\w+', text.lower())
  #return text

WORDS = Counter(words(open(txtSpellFile).read()))

def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

def spellCorrection(word): 
    """ Most probable spelling correction for word. """
    return max(candidates(word), key=P)
    #return word

def candidates(word): 
    """ Generate possible spelling corrections for word. """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    #return word

def known(words): 
    """ The subset of `words` that appear in the dictionary of WORDS. """
    return set(w for w in words if w in WORDS)
    #return words

def edits1(word):
    """ All edits that are one edit away from `word`. """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    """ All edits that are two edits away from `word`. """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Spell Correction End ###

### Replace Negations Begin ###

def replace(word, pos=None):
    """ Creates a set of all antonyms for the word and if there is only one antonym, it returns it """
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
      for lemma in syn.lemmas():
        for antonym in lemma.antonyms():
          antonyms.add(antonym.name())
    if len(antonyms) == 1:
      return antonyms.pop()
    else:
      return None

def replaceNegations(text):
    """ Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym """
    i, l = 0, len(text)
    words = []
    while i < l:
      word = text[i]
      if word == 'not' and i+1 < l:
        ant = replace(text[i+1])
        if ant:
          words.append(ant)
          i += 2
          continue
      words.append(word)
      i += 1
    return words

### Replace Negations End ###

def addNotTag(text):
	""" Finds "not,never,no" and adds the tag NEG_ to all words that follow until the next punctuation """
	return text
  #transformed = re.sub(r'\b(?:not|never|no)\b[\w\s]+[^\w\s]', 
  #     lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), 
  #     text,
  #     flags=re.IGNORECASE)
	#return transformed
  #return text

def addCapTag(word):
    """ Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_ """
    return word
    #if(len(re.findall("[A-Z]{3,}", word))):
    #    word = word.replace('\\', '' )
    #    transformed = re.sub("[A-Z]{3,}", "ALL_CAPS_"+word, word)
    #   return transformed
    #else:
    #    return word
    #return word


In [ ]:
""" Copyright 2017, Dimitrios Effrosynidis, All rights reserved. """


print("Starting preprocess..\n")

""" Tokenizes a text to its words, removes and replaces some of them """    
finalTokens = [] # all tokens
stoplist = stopwords.words('english')
my_stopwords = "multiexclamation multiquestion multistop url atuser st rd nd th am pm of for" # my extra stopwords
stoplist = stoplist + my_stopwords.split()
allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging
lemmatizer = WordNetLemmatizer() # set lemmatizer
#stemmer = PorterStemmer() # set stemmer

Starting preprocess..



# Loading the input file 

In [ ]:
csvDestinationFile = open(csvDestinationFileName, 'a+')
csvWriter = csv.writer(csvDestinationFile)

In [ ]:
csvDestinationFileName

'/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/PreProcessedPang-Ver2-00-11-27.csv'

In [ ]:
csvSourceFileName

'/MYDRIVE/My Drive/Colab Notebooks/SecondPaper/data/Pang-Ver2-00-10-24.csv'

In [ ]:
""" Copyright 2017, Dimitrios Effrosynidis, All rights reserved. """
print("Reading and loading dataset to memory")
#f = open("ss-twitterfinal.txt","r", encoding="utf8", errors='replace').read()
#df_original = pd.read_excel(DATASET_PATH, sheet_name="Stream2")
df_original = pd.read_csv(csvSourceFileName ,  header=None)
df_proccessed = df_original

print("The dataset is loaded")
print("the original dataset shape is",df_original.shape)

Reading and loading dataset to memory
The dataset is loaded
the original dataset shape is (2000, 3)


In [ ]:
df_proccessed.head(10)

,0,1,2
0,1,1,films adapted from comic books have had plenty...
1,2,1,every now and then a movie comes along from a ...
2,3,1,you ve got mail works alot better than it dese...
3,4,1,jaws is a rare film that grabs your atten...
4,5,1,moviemaking is a lot like being the general ma...
5,6,1,on june 30 1960 a self taught idealistic...
6,7,1,apparently director tony kaye had a major ba...
7,8,1,one of my colleagues was surprised when i told...
8,9,1,after bloody clashes and independence won lu...
9,10,1,the american action film has been slowly drown...


# Start to process 

In [ ]:
print("The processing tweets start...")
t0 = time()
totalSentences = 0
totalEmoticons = 0
totalSlangs = 0
totalSlangsFound = []
totalElongated = 0
totalMultiExclamationMarks = 0
totalMultiQuestionMarks = 0
totalMultiStopMarks = 0
totalAllCaps = 0

i = 0
for  row in df_proccessed.itertuples(index=False):
    t1 = time()  
    strTemp = row[2]
    onlyOneSentenceTokens = [] # tokens of one sentence each time
    
    try:
     strTemp = strTemp.encode('ascii', 'ignore').decode('unicode_escape')
    except: 
     pass
    
    strTemp = str(strTemp).lower()
    
     
    strTemp = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', strTemp) 
      
    strTemp = re.sub(r'[^\x00-\x7f]',r'',strTemp)
    
    strTemp = replaceURL(strTemp) # Technique 1
    
    strTemp = replaceAtUser(strTemp) # Technique 1
    
    strTemp = removeHashtagInFrontOfWord(strTemp) # Technique 1
    
    strTemp = removeNumbers(strTemp)
    
    strTemp = removeEmoticons(strTemp)
    #temp_slangs, temp_slangsFound = countSlang(strTemp)
    #totalSlangs += temp_slangs # total slangs for all sentences
    #for word in temp_slangsFound:
    #    totalSlangsFound.append(word) # all the slangs found in all sentences
    
    #strTemp = replaceSlang(strTemp) # Technique 2: replaces slang words and abbreviations with their equivalents
    strTemp = replaceContraction(strTemp) # Technique 3: replaces contractions to their equivalents
    strTemp = removeNumbers(strTemp) # Technique 4: remove integers from text

    emoticons = countEmoticons(strTemp) # how many emoticons in this sentence
    totalEmoticons += emoticons
    
    strTemp = removeEmoticons(strTemp) # removes emoticons from text

    
    #totalAllCaps += countAllCaps(strTemp)

    
    strTemp = replaceMultiExclamationMark(strTemp) # Technique 5: replaces repetitions of exlamation marks with the tag "multiExclamation"
    strTemp = replaceMultiQuestionMark(strTemp) # Technique 5: replaces repetitions of question marks with the tag "multiQuestion"
    strTemp = replaceMultiStopMark(strTemp) # Technique 5: replaces repetitions of stop marks with the tag "multiStop"
    
    
    translator = str.maketrans('', '', string.punctuation)
    strTemp = strTemp.translate(translator) # Technique 7: remove punctuation
    
    #totalElongated += countElongated(strTemp) # how many elongated words emoticons in this sentence
    

    tokens = nltk.word_tokenize(strTemp) # it takes a text as an input and provides a list of every token in it
    stopwords = stoplist
    tokens = [ w for w in tokens if not w in stopwords ]
### NO POS TAGGING BEGIN (If you don't want to use POS Tagging keep this section uncommented) ###
    
    for w in tokens:

        if (w not in stoplist): # Technique 10: remove stopwords
            #final_word = addCapTag(w) # Technique 8: Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_
            final_word = w.lower() # Technique 9: lowercases all characters
            final_word = replaceElongated(final_word) # Technique 11: replaces an elongated word with its basic form, unless the word exists in the lexicon
            #if len(final_word)>1:
            #    final_word = spellCorrection(final_word) # Technique 12: correction of spelling errors
            #final_word = lemmatizer.lemmatize(final_word) # Technique 14: lemmatizes words
            final_word = stemmer.stem(final_word) # Technique 15: apply stemming to words
            onlyOneSentenceTokens.append(final_word) 
    
    #tokens = [ w.lower() for w in tokens]
    #stopWords = set(stopwords.words('english'))

    
    
    onlyOneSentenceTokens = " ".join(onlyOneSentenceTokens) 

    csvWriter.writerow([ str(row[0]) , onlyOneSentenceTokens, str(row[1]) ])
    #csvWriter.writerow([  onlyOneSentenceTokens, str(row[0]) ])
    #i = i + 1
    


print("The processing tweets end")
    


The processing tweets start...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: invalid escape sequence '\ '


The processing tweets end


In [ ]:
csvDestinationFile.close()